In [1]:
import findspark
findspark.init()

from pyspark.sql import SparkSession
import collections

#Initializing SparkSession
spark = SparkSession.builder\
        .appName("Practice - 3")\
        .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/06/08 20:23:01 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/06/08 20:23:16 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


In [32]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, ArrayType

In [33]:
schema = StructType([
                    StructField("Id", IntegerType(), False)
                    , StructField("First_Name", StringType(), False)
                    , StructField("Last_Name", StringType(), False)
                    , StructField("url", StringType(), False)
                    , StructField("Published", StringType(), False)
                    , StructField("Hits", IntegerType(), False)
                    , StructField("Campaigns", ArrayType(StringType()), False)
                    ])

In [34]:
data = [
    [1, "Jules", "Damji", "https://tinyurl.1", "1/4/2016", 4535, ["twitter", "LinkedIn"]],
    [2, "Brooke","Wenig", "https://tinyurl.2", "5/5/2018", 8908, ["twitter", "LinkedIn"]], 
    [3, "Denny", "Lee", "https://tinyurl.3", "6/7/2019", 7659, ["web", "twitter", "FB", 
                                                                "LinkedIn"]],
    [4, "Tathagata", "Das", "https://tinyurl.4", "5/12/2018", 10568,["twitter", "FB"]],
    [5, "Matei","Zaharia", "https://tinyurl.5", "5/14/2014", 40578, ["web",
    "twitter", "FB", "LinkedIn"]],
    [6, "Reynold", "Xin", "https://tinyurl.6", "3/2/2015", 25568,["twitter", "LinkedIn"]]
        ]

In [35]:
blogs_df = spark.createDataFrame(data, schema)
blogs_df.show(10, truncate = False, vertical = False)

+---+----------+---------+-----------------+---------+-----+----------------------------+
|Id |First_Name|Last_Name|url              |Published|Hits |Campaigns                   |
+---+----------+---------+-----------------+---------+-----+----------------------------+
|1  |Jules     |Damji    |https://tinyurl.1|1/4/2016 |4535 |[twitter, LinkedIn]         |
|2  |Brooke    |Wenig    |https://tinyurl.2|5/5/2018 |8908 |[twitter, LinkedIn]         |
|3  |Denny     |Lee      |https://tinyurl.3|6/7/2019 |7659 |[web, twitter, FB, LinkedIn]|
|4  |Tathagata |Das      |https://tinyurl.4|5/12/2018|10568|[twitter, FB]               |
|5  |Matei     |Zaharia  |https://tinyurl.5|5/14/2014|40578|[web, twitter, FB, LinkedIn]|
|6  |Reynold   |Xin      |https://tinyurl.6|3/2/2015 |25568|[twitter, LinkedIn]         |
+---+----------+---------+-----------------+---------+-----+----------------------------+



In [36]:
print(blogs_df.printSchema())

root
 |-- Id: integer (nullable = false)
 |-- First_Name: string (nullable = false)
 |-- Last_Name: string (nullable = false)
 |-- url: string (nullable = false)
 |-- Published: string (nullable = false)
 |-- Hits: integer (nullable = false)
 |-- Campaigns: array (nullable = false)
 |    |-- element: string (containsNull = true)

None


In [37]:
print(blogs_df.schema)

StructType([StructField('Id', IntegerType(), False), StructField('First_Name', StringType(), False), StructField('Last_Name', StringType(), False), StructField('url', StringType(), False), StructField('Published', StringType(), False), StructField('Hits', IntegerType(), False), StructField('Campaigns', ArrayType(StringType(), True), False)])


In [38]:
from pyspark.sql.functions import *

In [39]:
blogs_df = blogs_df.withColumn("Published", to_date(col("Published"), 'M/d/yyyy'))

In [40]:
blogs_df.show()

+---+----------+---------+-----------------+----------+-----+--------------------+
| Id|First_Name|Last_Name|              url| Published| Hits|           Campaigns|
+---+----------+---------+-----------------+----------+-----+--------------------+
|  1|     Jules|    Damji|https://tinyurl.1|2016-01-04| 4535| [twitter, LinkedIn]|
|  2|    Brooke|    Wenig|https://tinyurl.2|2018-05-05| 8908| [twitter, LinkedIn]|
|  3|     Denny|      Lee|https://tinyurl.3|2019-06-07| 7659|[web, twitter, FB...|
|  4| Tathagata|      Das|https://tinyurl.4|2018-05-12|10568|       [twitter, FB]|
|  5|     Matei|  Zaharia|https://tinyurl.5|2014-05-14|40578|[web, twitter, FB...|
|  6|   Reynold|      Xin|https://tinyurl.6|2015-03-02|25568| [twitter, LinkedIn]|
+---+----------+---------+-----------------+----------+-----+--------------------+



In [41]:
blogs_df.select(year("Published")).distinct().orderBy(year("Published")).show(truncate = False)

+---------------+
|year(Published)|
+---------------+
|2014           |
|2015           |
|2016           |
|2018           |
|2019           |
+---------------+



In [43]:
blogs_df.select(year("Published").alias("Published_year"), month("Published"), day("Published")).show()

+--------------+----------------+--------------+
|Published_year|month(Published)|day(Published)|
+--------------+----------------+--------------+
|          2016|               1|             4|
|          2018|               5|             5|
|          2019|               6|             7|
|          2018|               5|            12|
|          2014|               5|            14|
|          2015|               3|             2|
+--------------+----------------+--------------+

